# AWS ASSIGNMENT>>>>

# 1.Explain the difference between AWS Regions, Availability Zones, and Edge Locations. Why is this important for
# data analysis and latency-sensitive applications?

In [ ]:
AWS Region:
A Region is a geographical area (like us-east-1, eu-west-1) containing multiple isolated data centers known as Availability Zones.

Each region is physically separated from others to ensure fault tolerance.

Availability Zone (AZ):
An Availability Zone is one or more discrete data centers within a region, each with redundant power, networking, and connectivity.

They are close enough for low-latency replication but isolated enough to protect applications from failures in other zones.

Edge Location:
Edge Locations are data centers that AWS uses to cache content closer to end-users.

Mainly used by services like Amazon CloudFront and AWS Global Accelerator to reduce latency in content delivery.
Fault Tolerance & High Availability: Deploying across multiple AZs or Regions ensures your data analytics apps stay up even if one data center fails.

Low Latency: Placing compute or caching layers (like CloudFront) near users or data sources improves real-time analytics performance.

Compliance: Some data must stay within specific countries or regions due to legal requirements.

Global Reach: You can deploy services closer to global users for better responsiveness.


# 2.Using the AWS CLI, list all available AWS regions. Share the command used and the output.

In [ ]:
aws ec2 describe-regions --all-regions --query "Regions[*].RegionName" --output table
----------------------------------
|        DescribeRegions         |
+--------------------------------+
|  af-south-1                   |
|  ap-east-1                    |
|  ap-south-1                   |
|  ap-northeast-1               |
|  ap-northeast-2               |
|  ap-northeast-3               |
|  ap-southeast-1               |
|  ap-southeast-2               |
|  ca-central-1                 |
|  eu-central-1                 |
|  eu-west-1                    |
|  eu-west-2                    |
|  eu-west-3                    |
|  eu-north-1                   |
|  eu-south-1                   |
|  me-south-1                   |
|  sa-east-1                    |
|  us-east-1                    |
|  us-east-2                    |
|  us-west-1                    |
|  us-west-2                    |
+--------------------------------+


# 3.Create a new IAM user with least privilege access to Amazon S3. Share your attached policies (JSON or
screenshot).

In [ ]:
aws iam create-user --user-name s3-readonly-user

{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "s3:ListBucket"
      ],
      "Resource": [
        "arn:aws:s3:::my-analytics-bucket"
      ]
    },
    {
      "Effect": "Allow",
      "Action": [
        "s3:GetObject"
      ],
      "Resource": [
        "arn:aws:s3:::my-analytics-bucket/*"
      ]
    }
  ]
}

aws iam put-user-policy \
  --user-name s3-readonly-user \
  --policy-name S3ReadOnlyPolicy \
  --policy-document file://s3_readonly_policy.json

# 4. Compare different Amazon S3 storage (Standard, Intelligent-Tiering, Glacier). When should each be used in data analytics workflows?


In [ ]:
| **Storage Class**       | **Use Case**                                                    | **Key Features**                                          |
| ----------------------- | --------------------------------------------------------------- | --------------------------------------------------------- |
| S3 Standard             | Frequently accessed data (active datasets)                      | Low latency, high throughput                              |
| S3 Intelligent-Tiering  | Unpredictable access patterns                                   | Automatic tiering; cost-optimized without performance hit |
| S3 Standard-IA          | Infrequently accessed data with quick retrieval needs           | Cheaper than Standard; charges per retrieval              |
| S3 One Zone-IA          | Infrequently accessed, non-critical data (can tolerate AZ loss) | Cheaper but only stored in one AZ                         |
| S3 Glacier              | Archived data, accessed occasionally (minutes to hours)         | Low cost, retrieval time in minutes/hours                 |
| S3 Glacier Deep Archive | Rarely accessed archives (e.g., compliance backups)             | Lowest cost, retrieval takes up to 12 hours               |


# 5. Create an S3 bucket and upload a sample dataset (CSV or JSON). Enable versioning and show at least two versions of one file.


In [ ]:
# Create bucket
aws s3api create-bucket --bucket analytics-sample-data --region us-east-1

# Enable versioning
aws s3api put-bucket-versioning --bucket analytics-sample-data --versioning-configuration Status=Enabled

# Upload v1 of file
aws s3 cp sample_data.csv s3://analytics-sample-data/sample_data.csv

# Modify file locally, then re-upload (v2)
echo "new,data,line" >> sample_data.csv
aws s3 cp sample_data.csv s3://analytics-sample-data/sample_data.csv

# List object versions
aws s3api list-object-versions --bucket analytics-sample-data --prefix sample_data.csv


# 6.Write and apply a lifecycle policy to move files to Glacier after 30 days and delete them after 90. Share the policy JSON or Screenshot.


In [ ]:
{
  "Rules": [
    {
      "ID": "MoveToGlacierAndDelete",
      "Status": "Enabled",
      "Prefix": "",
      "Transitions": [
        {
          "Days": 30,
          "StorageClass": "GLACIER"
        }
      ],
      "Expiration": {
        "Days": 90
      }
    }
  ]
}

aws s3api put-bucket-lifecycle-configuration \
  --bucket analytics-sample-data \
  --lifecycle-configuration file://lifecycle.json


# 7.Compare RDS, DynamoDB, and Redshift for use in different stages of a data pipeline. Give one use case for each.


In [ ]:
| **Service**  | **Best For**                                 | **Use Case**                                     |
| ------------ | -------------------------------------------- | ------------------------------------------------ |
| **RDS**      | Relational data, transactions                | Storing normalized app data (e.g., sales orders) |
| **DynamoDB** | Key-value or document-based, low-latency ops | Real-time leaderboard, IoT device states         |
| **Redshift** | Columnar, petabyte-scale analytics           | Running SQL queries on massive datasets          |


# 8.Create a DynamoDB table and insert 3 records manually. Then write a Lambda function that adds records when triggered by S3 uploads.


In [ ]:
aws dynamodb create-table \
  --table-name UploadEvents \
  --attribute-definitions AttributeName=FileName,AttributeType=S \
  --key-schema AttributeName=FileName,KeyType=HASH \
  --billing-mode PAY_PER_REQUEST

aws dynamodb put-item --table-name UploadEvents --item '{"FileName": {"S": "file1.csv"}}'
# Repeat for file2.csv to file5.csv

import boto3
import json

dynamodb = boto3.client('dynamodb')

def lambda_handler(event, context):
    for record in event['Records']:
        key = record['s3']['object']['key']
        dynamodb.put_item(
            TableName='UploadEvents',
            Item={'FileName': {'S': key}}
        )
    return {'statusCode': 200}


# 9.What is serverless computing? Discuss pros and cons of using AWS Lambda for data pipelines.

In [ ]:
#Definition:
Serverless computing means running code without managing servers — you only pay for what you use.

#Pros:
Auto scaling

No server management

Cost-effective for infrequent jobs

#Cons:
Timeout limits (15 minutes for Lambda)

Cold start latency

Complex workflows require orchestration (e.g., Step Functions)

# 10.Create a Lambda function triggered by S3 uploads that logs file name, size, and timestamp to Cloudwatch. Share code and a log screenshot?


import logging

def lambda_handler(event, context):
    for record in event['Records']:
        key = record['s3']['object']['key']
        size = record['s3']['object']['size']
        time = record['eventTime']
        logging.info(f"File uploaded: {key}, Size: {size}, Time: {time}")
    return {'statusCode': 200}


# 11.Use AWS Glue to crawl your S3 dataset, create a Data Catalog table, and run a Glue job to convert CSV data to parquet. Share job code and output location.


We will:

Set up an S3 bucket and upload CSV data

Create a Glue Crawler to detect schema and create a Data Catalog table

Write a Glue Job (Spark script) to convert CSV to Parquet

Output the Parquet files to a new S3 location

aws s3 cp sample_data.csv s3://analytics-sample-data/raw/

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

# Boilerplate
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Load CSV table from Glue Catalog
datasource = glueContext.create_dynamic_frame.from_catalog(
    database="analytics_db", 
    table_name="raw_sample_data", 
    transformation_ctx="datasource"
)

# Write to Parquet format
glueContext.write_dynamic_frame.from_options(
    frame=datasource,
    connection_type="s3",
    connection_options={"path": "s3://analytics-sample-data/parquet/"},
    format="parquet"
)

job.commit()

Replace "raw_sample_data" with the actual table name the crawler generated.

# 12.Explain the difference between Kinesis Data Streams, Kinesis Firehose, and Kinesis Data Analytics. Provide a
real-world example of how each would be used.


#1.Kinesis Data Streams (KDS)
What it is: A real-time data streaming service that lets you collect, process, and store large streams of data records (millisecond latency).

#Use case: Best for custom processing logic using AWS Lambda, EC2, or Kinesis Client Library.

Example:
A ride-hailing app streams GPS data from drivers to Kinesis Data Streams, which triggers a Lambda to calculate traffic heatmaps in real-time.

#2.Kinesis Data Firehose
What it is: A fully managed data delivery service that automatically loads streaming data to destinations like S3, Redshift, or Elasticsearch.

No manual provisioning needed.

Near real-time, but with buffering (up to 60 secs or 1MB).

Example:
A financial app sends transaction logs to Kinesis Firehose, which automatically stores the data in S3 in Parquet format, then loads it into Redshift for reporting.

#3. Kinesis Data Analytics
What it is: A service to run SQL or Apache Flink on streaming data in real time (from Kinesis Data Streams or Firehose).

Great for real-time filtering, aggregations, or joins without building code.

Example:
A logistics company uses Kinesis Data Analytics to detect temperature anomalies from IoT sensor data streaming in via Data Streams, triggering alerts when thresholds are exceeded.



# 13.What is columnar storage and how does it benefit Redshift performance for analytics workloads"


How Columnar Storage Works in Redshift:
Redshift stores data by columns.

Instead of reading an entire row, Redshift reads only the relevant columns needed for a query.

This reduces the amount of data that needs to be scanned, improving I/O efficiency.

Benefits of Columnar Storage for Redshift Analytics Workloads:
Efficient Data Scanning:

Analytics queries often access only a subset of columns.

Columnar storage means Redshift reads only those columns, reducing disk I/O.

Better Compression:

Data within a column tends to be of the same data type and often has similar values.

This makes compression more effective, reducing storage size and improving query speed.

Faster Aggregations and Computations:

Many analytics operations (e.g., SUM, AVG, COUNT) operate on columns.

Columnar storage allows for optimized vectorized processing.

Improved Query Performance:

By avoiding unnecessary data retrieval and leveraging compression, Redshift executes analytic queries faster and more efficiently.


# 14.Load a CSV file from S3 into Redshift using the COPY command. Share table schema, command used, and sample output from a query.



1. Table Schema
Let's say we have a CSV file with user data like this:

user_id	first_name	last_name	email	signup_date
1	Alice	Smith	alice@example.com	2023-01-15
2	Bob	Johnson	bob@example.com	2023-02-20

We create a table in Redshift to store this data:

CREATE TABLE users (
    user_id INT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    signup_date DATE
);

COPY users
FROM 's3://mybucket/data/users.csv'
IAM_ROLE 'arn:aws:iam::123456789012:role/MyRedshiftRole'
CSV
IGNOREHEADER 1
DATEFORMAT 'auto';

SELECT * FROM users;


# 15.What is the role of the AWS Glue Data Catalog in Athena? How does schema-on-read work?

Role of AWS Glue Data Catalog in Athena:
AWS Glue Data Catalog acts as a central metadata repository for Athena.

It stores information about your data sources such as databases, tables, schemas, and partitions.

When you run a query in Athena, it refers to the Glue Data Catalog to understand the structure of the data (table schema, column types, location, etc.).

Glue Data Catalog enables Athena to query data directly from sources like S3 without moving or transforming the data beforehand.

It also integrates with other AWS analytics services, providing a unified metadata layer.

How Schema-on-Read Works:
Traditional databases use schema-on-write: you define the schema before loading data, and data must conform to it.

Athena uses schema-on-read, meaning:

You define the schema when you run the query, not before loading the data.

Athena reads the raw data stored (e.g., in S3) and applies the schema dynamically at query time.

This allows querying of semi-structured or raw data without upfront transformation.

Benefits of schema-on-read:

Flexibility to query data in its original format.

No need to preprocess or restructure data before analysis.

Easy to adapt to changing data formats or new data sources.


# 16.Create an Athena table from S3 data using Glue Catalog. Run a query and share the SQL + result screenshot

In [ ]:
# <!-- Step 1: Prepare Your Data in S3
# Assume you have a CSV file stored in S3 at:

s3://mybucket/data/sales_data.csv
Sample data format (CSV):

# order_id	product	quantity	price	order_date
# 101	Widget A	5	20.5	2023-04-01
# 102	Widget B	3	15.0	2023-04-02

# Step 2: Create Glue Crawler to Catalog the Data
# Go to AWS Glue Console → Crawlers → Add crawler.
 -->
Specify the S3 path (s3://mybucket/data/).

# Choose an existing or create a new database, e.g., sales_db.

# Run the crawler; it will scan the data and create a table like sales_data with the schema inferred.

# Step 3: Verify the Table in Glue Data Catalog
# The Glue Data Catalog now has a database sales_db and table sales_data.

# It contains schema info (columns, types).

# Step 4: Query the Table in Athena
# Open Amazon Athena Console and set the database:

USE sales_db;
# Run a sample query, for example:


SELECT product, SUM(quantity) AS total_quantity, SUM(quantity * price) AS total_revenue
FROM sales_data
GROUP BY product
ORDER BY total_revenue DESC;

# Step 5: Capture and Share
# Take a screenshot of the Athena query editor showing the SQL above.

# Also, capture the query results showing aggregated total quantity and revenue per product.

# Example SQL for Submission:

USE sales_db;

SELECT product, 
       SUM(quantity) AS total_quantity, 
       SUM(quantity * price) AS total_revenue
FROM sales_data
GROUP BY product
ORDER BY total_revenue DESC;


# 17 Describe how Amazon Quicksight supports business intelligence in a serverless data architecture. What are SPICE and embedded dashboards#

<!-- How Amazon QuickSight Supports Business Intelligence in Serverless Data Architecture
Serverless BI Service:
QuickSight is a fully managed, serverless business intelligence (BI) service by AWS. This means there’s no infrastructure to manage, scale, or patch — AWS handles all backend resources automatically.

Seamless Integration with AWS Data Sources:
It connects directly to various AWS data services like Athena, Redshift, S3 (via Athena or Redshift Spectrum), RDS, and others, enabling quick access to data stored in the cloud without moving or copying it.

Scalable and Cost-Effective:
Because it’s serverless, QuickSight scales automatically to support any number of users, from a few analysts to thousands of readers, and charges are usage-based — you pay only for what you use.

Fast, Interactive Analytics:
Provides interactive dashboards, visualizations, and ad hoc querying with minimal latency, supporting data-driven decision-making.

Secure and Compliant:
Integrates with AWS IAM and encryption features, providing secure data access and governance.

What is SPICE?
SPICE stands for Super-fast, Parallel, In-memory Calculation Engine.

It is QuickSight’s in-memory data engine that allows rapid data retrieval and interactive analytics.

SPICE imports data from various sources and stores it in a highly optimized columnar format in memory.

This accelerates query performance, enabling fast dashboards and visualizations even on large datasets.

Users can refresh SPICE datasets on a schedule to keep data up-to-date.

What are Embedded Dashboards?
Embedded dashboards allow you to integrate QuickSight visuals and reports into your own applications or websites.

Using APIs and SDKs, you can securely embed interactive dashboards for your customers or internal users without requiring them to log into QuickSight separately.

This enables seamless BI experiences within custom portals or SaaS products.

Supports fine-grained access control so different users see only data relevant to them.

 -->

# 18.Connect Quicksight to Athena or Redshift and build a dashboard with at least one calculated field and one filter. Share a screenshot of your final dashboard.

In [ ]:
Step 1: Connect QuickSight to Data Source
For Athena:

In QuickSight console, click Manage data → New data set.

Select Athena as the data source.

Provide a data source name and choose the AWS region.

Choose the database and table (or write a custom query).

Import data or use Direct Query mode.

For Redshift:

In QuickSight console, click Manage data → New data set.

Select Redshift as the data source.

Provide Redshift cluster endpoint, database, username, and password.

Choose the table or write a custom SQL query.

Step 2: Create Dataset and Prepare Data
After connecting, QuickSight imports or queries the data.

You can modify data types and rename columns if needed.

Step 3: Create a Calculated Field
In the dataset, click Add calculated field.

Example:
If you have sales data, create a field like Total Revenue = quantity_sold * price_per_unit.

Enter the formula and save.

Step 4: Build the Dashboard
Click New analysis, choose your dataset.

Add visuals like bar charts, tables, or pie charts.

Add your calculated field as a metric.

Add a filter to the dashboard. For example:

Filter by date range.

Filter by category or region.

Step 5: Save and Share
Save your analysis.

Take a screenshot of the dashboard showing:

At least one visual using the calculated field.

The filter control panel visible.


# 19.Explain how AWS CloudWatch and CloudTrail differ. IN a data analytics pipeline, what role does each play in monitoring, auditing, and troubleshooting.


In [ ]:
# AWS CloudWatch -->
# Purpose:
# CloudWatch is primarily a monitoring and observability service.

# What it does:

# Collects and tracks metrics (CPU usage, memory, disk I/O, network traffic) from AWS resources and applications.

# Captures logs from services like Lambda, EC2, Redshift, etc.

# Allows setting alarms to notify you about performance or operational issues.

# Provides dashboards and visualizations for real-time monitoring.

# Role in Data Analytics Pipeline:

# Monitors health and performance of data processing components (e.g., Redshift clusters, Glue jobs, Lambda functions).

# Tracks resource utilization to optimize capacity and cost.

# Helps troubleshoot failures or bottlenecks by analyzing logs and metrics.

# AWS CloudTrail
# Purpose:
# CloudTrail is a governance, compliance, and auditing service.

# What it does:

# Records API calls and user activity across your AWS account.

# Tracks who did what, when, and from where.

# Stores logs of management and data plane events for security auditing and forensic analysis.

# Role in Data Analytics Pipeline:

# Audits user and system actions on data resources (e.g., who created or modified a Glue job, queried Athena, or accessed S3 buckets).

# Ensures compliance with security policies by tracking unauthorized or unusual activities.

# Provides audit trails necessary for regulatory requirements. -->

# 20.Describe a complete end-to-end data analytics pipeline using AWS services. Include services for data ingestion, storage, transformation, querying, and visualization. (Example: S3 → Lambda → Glue → Quicksight)

Example Pipeline:
Data Ingestion → Storage → Transformation → Querying → Visualization

1. Data Ingestion: AWS Kinesis or AWS Lambda
AWS Kinesis Data Streams
For real-time streaming data ingestion from sources such as IoT devices, application logs, or clickstreams. Kinesis collects and buffers streaming data continuously.

AWS Lambda
For serverless event-driven ingestion, e.g., processing uploaded files in S3 or API events.

2. Data Storage: Amazon S3
Amazon S3 serves as a central data lake storing raw and processed data.

S3 is durable, scalable, and cost-effective, perfect for storing large volumes of structured and unstructured data.

Raw data ingested from Kinesis or Lambda is saved here.

3. Data Transformation: AWS Glue
AWS Glue performs serverless ETL (Extract, Transform, Load).

Glue Crawlers scan the raw data in S3, automatically infer schemas, and populate the Glue Data Catalog.

Glue ETL jobs clean, enrich, and transform the raw data into a structured format optimized for analytics.

Transformed data is saved back into S3 in a columnar format like Parquet for efficient querying.

4. Querying: Amazon Athena or Amazon Redshift
Amazon Athena
Serverless, interactive SQL querying directly on data stored in S3 using the Glue Data Catalog metadata. Ideal for ad hoc queries without needing to move data.

Amazon Redshift
For high-performance data warehousing and complex analytics, where data can be loaded from S3 or queried using Redshift Spectrum directly on S3 data.

5. Visualization: Amazon QuickSight
Amazon QuickSight connects to Athena, Redshift, or other data sources to create interactive dashboards and visualizations.

It enables business users and analysts to explore data, generate insights, and share reports.


# 21.Explain why you would choose each service for the stage it’s used in4
# _ Submission Instructions Format
# One .docx or .pdf file containing
# _ Written responses to theory question<
# _ Screenshots and code snippets for hands-on task<
# _ Clear question numbers and titles

When designing a data workflow or pipeline, selecting the right AWS service for each stage depends on the specific requirements of data ingestion, storage, processing, and analysis. Here’s a typical breakdown:

1. Data Ingestion: AWS Kinesis / AWS Glue / AWS Data Pipeline
Why choose?

AWS Kinesis: Real-time streaming data ingestion, low latency.

AWS Glue: Serverless ETL to crawl and catalog data for batch ingestion.

AWS Data Pipeline: Orchestrate and automate data movement and transformation on schedules.

2. Data Storage: Amazon S3 / Amazon Redshift / Amazon RDS
Why choose?

Amazon S3: Cost-effective, durable, scalable object storage for raw or processed data lakes.

Amazon Redshift: Fully managed, high-performance data warehouse optimized for complex analytical queries.

Amazon RDS: Managed relational database for transactional workloads with structured data.

3. Data Processing & Transformation: AWS Glue / AWS EMR / AWS Lambda
Why choose?

AWS Glue: Serverless ETL with built-in data catalog integration, ideal for batch transformations.

AWS EMR: Managed Hadoop/Spark clusters for big data processing, custom analytics workloads.

AWS Lambda: Event-driven serverless functions for lightweight, real-time data processing.

4. Data Analytics & Visualization: Amazon Athena / Amazon QuickSight / Amazon Redshift Spectrum
Why choose?

Amazon Athena: Serverless interactive querying directly on data in S3 using SQL, great for ad hoc analysis.

Amazon QuickSight: Scalable business intelligence tool for creating visual dashboards and reports.

Redshift Spectrum: Extends Redshift to query data directly in S3 without loading it, combining warehouse and data lake.

